<a href="https://colab.research.google.com/github/sumathi553/Assignment-3/blob/main/assignment_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Assignment 4

Assignment date: 19.02.2002

Student name: D.sumathi

Student roll number: 961919104701

Maximum marks: 2

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import pad_sequences
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping

In [4]:
data=pd.read_csv('/content/spam.csv',encoding='latin')

In [5]:
df = pd.read_csv('/content/spam.csv',delimiter=',',encoding='latin-1')
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [6]:
df.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'],axis=1,inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   v1      5572 non-null   object
 1   v2      5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [7]:
X = df.v2
Y = df.v1
le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)

In [9]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15)

In [10]:
max_words = 1000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = pad_sequences(sequences,maxlen=max_len)

In [11]:
inputs = Input(name='inputs',shape=[max_len])
layer = Embedding(max_words,50,input_length=max_len)(inputs)
layer = LSTM(64)(layer)
layer = Dense(256,name='FC1')(layer)
layer = Activation('relu')(layer)
layer = Dropout(0.5)(layer)
layer = Dense(1,name='out_layer')(layer)
layer = Activation('sigmoid')(layer)
model = Model(inputs=inputs,outputs=layer)


In [13]:
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

In [14]:
model.fit(sequences_matrix,Y_train,batch_size=128,epochs=10,
          validation_split=0.2)

Epoch 1/10
30/30 [==============================] - 8s 30ms/step - loss: 0.3014 - accuracy: 0.8949 - val_loss: 0.1681 - val_accuracy: 0.9726
Epoch 2/10
30/30 [==============================] - 0s 13ms/step - loss: 0.0704 - accuracy: 0.9828 - val_loss: 0.0632 - val_accuracy: 0.9810
Epoch 3/10
30/30 [==============================] - 0s 13ms/step - loss: 0.0365 - accuracy: 0.9886 - val_loss: 0.0587 - val_accuracy: 0.9821
Epoch 4/10
30/30 [==============================] - 0s 13ms/step - loss: 0.0309 - accuracy: 0.9926 - val_loss: 0.0863 - val_accuracy: 0.9831
Epoch 5/10
30/30 [==============================] - 0s 13ms/step - loss: 0.0220 - accuracy: 0.9952 - val_loss: 0.0771 - val_accuracy: 0.9831
Epoch 6/10
30/30 [==============================] - 0s 13ms/step - loss: 0.0185 - accuracy: 0.9955 - val_loss: 0.0615 - val_accuracy: 0.9778
Epoch 7/10
30/30 [==============================] - 0s 14ms/step - loss: 0.0144 - accuracy: 0.9950 - val_loss: 0.0686 - val_accuracy: 0.9821
Epoch 8/10
30

In [15]:
model.save('sms_classifier.h5')

In [16]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = pad_sequences(test_sequences,maxlen=max_len)

In [17]:
accr = model.evaluate(test_sequences_matrix,Y_test)

27/27 [==============================] - 0s 6ms/step - loss: 0.0914 - accuracy: 0.9880


In [18]:
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

Test set
  Loss: 0.091
  Accuracy: 0.988
